In [1]:
import pandas as pd 
import polars as pl
import numpy as np
import xarray as xr
from datetime import datetime, timezone

In [2]:
dep = "RS03AXBS-LJ03A-09-HYDBBA302_5"
folder ="rca_correction_cals"

In [3]:
ds_original = xr.open_dataset(f"../metadata/{folder}/{dep}.nc")

In [4]:
ds_original

<xarray.Dataset>
Dimensions:         (frequency: 20)
Coordinates:
  * frequency       (frequency) float64 1e+04 2.01e+04 ... 1.907e+05 2e+05
Data variables:
    sensitivity_0   (frequency) float64 ...
    sensitivity_90  (frequency) float64 ...
    sensitivity     (frequency) float64 ...
Attributes:
    asset_id:          ATOSU-58324-00016
    model:             SB35-ETH
    serial_number:     1412
    calibration_date:  2018-02-02T00:00:00.000000Z

In [5]:
# Get the sensitivity value at the 0th index
sensitivity0_at_0 = ds_original["sensitivity_0"].isel(frequency=0).item()
sensitivity90_at_0 = ds_original["sensitivity_90"].isel(frequency=0).item()
sensitivity_at_0 = ds_original["sensitivity"].isel(frequency=0).item()
# Create a new dataset for the 0 frequency point
ds_new_point = xr.Dataset(
    {
        "sensitivity_0": ("frequency", np.array([sensitivity0_at_0])),
        "sensitivity_90": ("frequency", np.array([sensitivity90_at_0])),
        "sensitivity": ("frequency", np.array([sensitivity_at_0])),
    },
    coords={
        "frequency": np.array([0.0]),
    },
)


In [6]:
ds_new_point

<xarray.Dataset>
Dimensions:         (frequency: 1)
Coordinates:
  * frequency       (frequency) float64 0.0
Data variables:
    sensitivity_0   (frequency) float64 -168.4
    sensitivity_90  (frequency) float64 -168.9
    sensitivity     (frequency) float64 -39.75

In [7]:
# Concatenate the new dataset to the front of the original dataset
ds_concatenated = xr.concat([ds_new_point, ds_original], dim="frequency")
ds_concatenated

<xarray.Dataset>
Dimensions:         (frequency: 21)
Coordinates:
  * frequency       (frequency) float64 0.0 1e+04 2.01e+04 ... 1.907e+05 2e+05
Data variables:
    sensitivity_0   (frequency) float64 -168.4 -168.4 -171.2 ... -168.5 -171.3
    sensitivity_90  (frequency) float64 -168.9 -168.9 -170.6 ... -169.4 -173.2
    sensitivity     (frequency) float64 -39.75 -39.75 -42.0 ... -40.05 -43.35

In [8]:
ds_concatenated.to_netcdf(f"../metadata/{folder}/whoops/{dep}.nc")

In [9]:
tuple([
    10,
    30000
  ])

(10, 30000)